Construindo a Rede Neura
========================

Redes neurais são compostas por camadas/módulos que realizam operações em dados. 
O namespace [torch.nn](https://pytorch.org/docs/stable/nn.html) fornece todos 
os blocos de construção necessários para construir sua própria rede neural. 
Cada módulo no PyTorch é uma subclasse de
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
Essa estrutura aninhada permite construir e gerenciar arquiteturas complexas facilmente.

Nas seções a seguir, construiremos uma rede neural para classificar imagens no conjunto de dados FashionMNIST.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Obtenha o dispositivo para treinamento
=======================

Queremos poder treinar nosso modelo em um
[acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA ou XPU. Se o acelerador atual estiver disponível, 
o usaremos. Caso contrário, usaremos a CPU.


In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu" # type: ignore
#device = "cuda" if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Defina a Classe
================

Definimos nossa rede neural subclassificando  `nn.Module`, 
e inicializando as camadas da rede neural em `__init__`. Cada subclass `nn.Module`
implementa as operações nos dados de entrada do método `forward`.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Criamos uma instância de `NeuralNetwork`, movemos para o `device`,
e imprimimos sua estrutura.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Para usar o modelo, passamos os dados de entrada. Isso executa o
`forward`, juntamente com algumas 
[operações em segundo plano ](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Não chame `model.forward()` diretamente!

A chamada do modelo na entrada retorna um tensor bidimensional 
com dim=0 correspondendo a cada saída de 10 valores brutos 
previstos para cada classe, e dim=1 correspondendo aos valores 
individuais de cada saída. Obtemos as probabilidades de previsão 
passando-as por uma instância do módulo `nn.Softmax`.

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

------------------------------------------------------------------------


Camadas do modelo
============

Vamos analisar as camadas do modelo FashionMNIST. Para ilustrar, pegaremos um minilote de amostra com 3 imagens de tamanho 28x28 e veremos o que acontece com ele à medida que o passamos pela rede.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

nn.Flatten
==========

Inicializamos a camada 
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
para converter cada imagem 2D 28x28 em uma matriz contígua de 784 
valores de pixels (a dimensão do minibatch (em dim=0) é mantida).

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

nn.Linear
=========

A [Camada linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
é um módulo que aplica uma transformação linear na entrada usando seus pesos e vieses armazenados.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

nn.ReLU
=======

Ativações não lineares são o que criam os mapeamentos complexos 
entre as entradas e saídas do modelo. Elas são aplicadas após 
transformações lineares para introduzir a **não linearidade**, 
ajudando as redes neurais a aprender uma ampla variedade de fenômenos.

Neste modelo, usamos
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
entre nossas camadas lineares, mas há outras ativações 
para introduzir não linearidade em seu modelo.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
 é um contêiner ordenado de módulos. Os dados são passados ​​por todos os módulos na mesma ordem definida. Você pode usar contêineres sequenciais para montar uma rede rápida como `seq_modules`.


In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

A última camada linear da rede neural retorna `logits` valores brutos em
 \[-infty, infty\] que são passados ​​para o módulo
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html).
Os logits são escalados para valores [0, 1] que representam as probabilidades previstas do modelo para cada classe. O parâmetro `dim`, indica a dimensão ao longo da qual os valores devem somar 1.


In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Parâmetros do modelo
================

Muitas camadas dentro de uma rede neural são parametrizadas, ou seja, 
possuem pesos e vieses associados que são otimizados durante o treinamento.
A subclassificação `nn.Module` rastreia automaticamente todos os campos 
definidos dentro do objeto do seu modelo e torna todos os parâmetros 
acessíveis usando os método `parameters()` ou `named_parameters()`.

Neste exemplo, iteramos sobre cada parâmetro e imprimimos seu tamanho e uma prévia de seus valores.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

------------------------------------------------------------------------


Leitura adicional
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
